In [1]:
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import *

# from pyspark.sql import functions as sf

spark = SparkSession.builder \
    .appName("sql2") \
    .getOrCreate()

23/11/11 22:21:39 WARN Utils: Your hostname, kenans-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.102 instead (on interface en0)
23/11/11 22:21:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Exception in thread "main" java.lang.ExceptionInInitializerError
	at org.apache.spark.unsafe.array.ByteArrayMethods.<clinit>(ByteArrayMethods.java:54)
	at org.apache.spark.internal.config.package$.<init>(package.scala:1006)
	at org.apache.spark.internal.config.package$.<clinit>(package.scala)
	at org.apache.spark.deploy.SparkSubmitArguments.$anonfun$loadEnvironmentArguments$3(SparkSubmitArguments.scala:157)
	at scala.Option.orElse(Option.scala:447)
	at org.apache.spark.deploy.SparkSubmitArguments.loadEnvironmentArguments(SparkSubmitArguments.scala:157)
	at org.apache.spark.deploy.SparkSubmitArguments.<init>(SparkSubmitArguments.scala:115)
	at org.apache.spark.deploy.SparkSubmit$$anon$2$$anon$3.<init>(SparkSubmit.scala:990)
	at org.apache.spar

Exception: Java gateway process exited before sending its port number

In [2]:
peopleV2 = spark.createDataFrame([Row(name="andrew", age=32),
                                  Row(name="james", age=23),
                                  Row(name="james", age=4),
                                  Row(name="ann", age=12),
                                  Row(name="Alice", age=2),
                                  Row(name="Alice", age=3),
                                  Row(name="Alice", age=6),
                                  Row(name="Bob", age=5),
                                  Row(name="Bob", age=10),
                                  Row(name="Bob", age=12),
                                  Row(name="Cathy", age=7),
                                  Row(name="Cathy", age=8)])

peopleV2.groupby('name') \
    .agg(count('*').alias('count')) \
    .sort(col('count').desc()) \
    .show()

+------+-----+
|  name|count|
+------+-----+
| Alice|    3|
|   Bob|    3|
| james|    2|
| Cathy|    2|
|andrew|    1|
|   ann|    1|
+------+-----+


In [3]:
peopleV2.groupby('name') \
    .agg({"*": 'count'}) \
    .withColumnRenamed('count(1)', 'count') \
    .sort('count', ascending=True).show()

+------+-----+
|  name|count|
+------+-----+
|andrew|    1|
|   ann|    1|
| james|    2|
| Cathy|    2|
| Alice|    3|
|   Bob|    3|
+------+-----+


In [4]:
peopleV2.groupby('name') \
    .agg(count('*').alias('count')) \
    .sort(col('count').asc()) \
    .show()

+------+-----+
|  name|count|
+------+-----+
|andrew|    1|
|   ann|    1|
| james|    2|
| Cathy|    2|
| Alice|    3|
|   Bob|    3|
+------+-----+


In [5]:
peopleV2.groupby('name') \
    .agg(min('age').alias('min_age'), max('age').alias('max_age')) \
    .sort(col('min_age').asc()) \
    .show()

+------+-------+-------+
|  name|min_age|max_age|
+------+-------+-------+
| Alice|      2|      6|
| james|      4|     23|
|   Bob|      5|     12|
| Cathy|      7|      8|
|   ann|     12|     12|
|andrew|     32|     32|
+------+-------+-------+


In [8]:
@pandas_udf('int', PandasUDFType.GROUPED_AGG)
def min_udf(v):
    return v.min()


peopleV2.groupby('name').agg(min_udf(peopleV2.age).alias('min_age')).sort('name').show()

+------+-------+
|  name|min_age|
+------+-------+
| Alice|      2|
|   Bob|      5|
| Cathy|      7|
|andrew|     32|
|   ann|     12|
| james|      4|
+------+-------+


/Users/kenanhancer/Documents/projects/python-projects-kenanhancer/python_pyspark_demo/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/pandas/serializers.py:224: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
/Users/kenanhancer/Documents/projects/python-projects-kenanhancer/python_pyspark_demo/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/pandas/serializers.py:224: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
/Users/kenanhancer/Documents/projects/python-projects-kenanhancer/python_pyspark_demo/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/pandas/serializers.py:224: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
/Users/ken